In [ ]:
# bbconfig2original config
import numpy as np
import json
import os
from typing import Dict, List, Tuple, Any
from biotite.structure.io import load_structure
from biotite.structure import AtomArray
import biotite.structure as struc

from protenix.data.constants import mmcif_restype_3to1 as AA_3TO1
PROTEIN_REP_RESIDUE = 'GLY'

output_json = []
with open('./../examples/bb_generation.json', 'r') as f:
    bb_configs = json.load(f)
    for i in range(len(bb_configs)):
        bb_config = bb_configs[i]
        bb_config['fixed'] = bb_config['fixed'].split(',')
        bb_config['fixed_type'] = bb_config['fixed_type'].split(', ')
        for fixed_residue, bb_atom in bb_config['fixed_atom'].items():
            bb_config['fixed_atom'][fixed_residue] = bb_atom.split(',')
        assert len(bb_config['fixed']) == len(bb_config['fixed_type'])
        stack = load_structure(bb_config['pdb'])

        infer_seq = []
        output_dict = {'sequences':[],'name':''}

        for j in range(len(bb_config['fixed'])):
            if bb_config['fixed_type'][j] == 'proteinbb':
                min_len,max_len = map(int, bb_config['fixed'][j].split('-'))
                assert min_len <= max_len
                # randomly select a length between min_len and max_len
                length = np.random.randint(min_len, max_len + 1)
                for _ in range(length):
                    infer_seq.append(AA_3TO1[PROTEIN_REP_RESIDUE])
            elif bb_config['fixed_type'][j] == 'proteinChain':
                chain_id = bb_config['fixed'][j].split('-')[0][0]
                res_id = int(bb_config['fixed'][j].split('-')[1])
                mask = (stack.chain_id == chain_id) & (stack.res_id == res_id)
                res_name = stack[np.where(mask)[0][0]].res_name
                infer_seq.append(AA_3TO1[res_name])
        infer_seq = ''.join(infer_seq)
        output_dict['name'] = bb_config['name']
        output_dict['sequences'].append({
            "proteinChain":{
                "sequence":infer_seq,
                "count": 1
            }
        })
        output_json.append(output_dict)

output_path = './../examples/bb_generation_converted.json'
with open(output_path, 'w') as f:
    json.dump(output_json, f, indent=2)

print(f"Successfully converted {len(output_json)} configurations")
print(f"Output saved to: {output_path}")

if output_json:
    print("\nExample converted config:")
    print(json.dumps(output_json[0], indent=2))

: 

In [ ]:
with open('./../examples/bb_generation_converted.json', "r") as f:
    inputs = json.load(f)
inputs

In [ ]:
output_json